In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
max_cpu = cpu_count()

In [74]:
from sklearn.linear_model import LinearRegression, LassoCV

def np_standardize(arr):
    return (arr - np.mean(arr,axis=0))/np.std(arr,axis=0)

def postlassoIV_est(y,d,w,Z):
    '''
    Post Lasso IV Estimator based on Section 2.4 of BCCH 2015. Expects numpy arrays as arguments.
    For our setting, we only have 1 endogenous regression X which enters as a column vector.
    
    Parameters:
        y - outcome vector
        d - endogenous regressor
        w - exogenous regressors (excluded instruments)
        Z - instruments for D (included instruments)
    '''
    Z_standard = np_standardize(Z)
    
    # constructing optimal instrument \hat{D}
    ## selecting instruments
    lasso = LassoCV()
    lasso.fit(np.concatenate((Z_standard,w), axis=1),d)
    var_select = [i for i, coef in enumerate(lasso.coef_.flatten()[:Z.shape[1]]) if coef != 0]
    Z_select = Z[:,var_select]
    
    ## getting \hat{D} using fitted values of OLS of d on Z_select
    lr = LinearRegression()
    lr.fit(np.concatenate((Z_select,w), axis=1),d)
    D_hat = lr.predict(np.concatenate((Z_select,w), axis=1)).T
    
    # calculating IV estimator (since we are in the just identified case due to having one endogenous regressor, the
    # standard covariance formula suffices
    #alpha = np.linalg.inv(D_hat @ d.reshape(d.shape[0],1)) @ D_hat.T @ y
    alpha = np.cov(D_hat,y)[0,1] / np.cov(D_hat,d)[0,1]
    
    return alpha

def postlassoIV_robustvar(y,d,w,Z):
    pass

In [75]:
df = pd.read_csv('data/final_data.csv')

In [76]:
y = df['lsales'].to_numpy()
d = df['lprice'].to_numpy()
w = df.filter(regex='^cntyfe_|^timefe_').to_numpy()
Z = df.filter(regex='^aftexpl\\.dist_to_ref\\d+').to_numpy()

In [77]:
Z_standard = np_standardize(Z)
    
# constructing optimal instrument \hat{D}
## selecting instruments
lasso = LassoCV()
lasso.fit(np.concatenate((Z_standard,w), axis=1),d)
var_select = [i for i, coef in enumerate(lasso.coef_.flatten()[:Z.shape[1]]) if coef != 0]
Z_select = Z[:,var_select]

## getting \hat{D} using fitted values of OLS of d on Z_select
lr = LinearRegression()
lr.fit(np.concatenate((Z_select,w), axis=1),d)
D_hat = lr.predict(np.concatenate((Z_select,w), axis=1)).T

In [78]:
postlassoIV_est(y,d,w,Z)

-0.6128438965704044